In [11]:
import numpy as np

import matplotlib.pyplot as plt

import sys
sys.path.insert(0,"C:/Users/Dustin/Desktop/Numerische Methoden/VS Code/NM-Code-for-Excercises")
import shortcuts as s

In [12]:
def f(x):
    return (x**4)*(np.log10(x+(1+x**2)**0.5))          #passt, habs gecheckt, f ist ein numpyfloat, ich gehe davon aus, dass mit log, log10 gemeint ist

In [13]:
#Extended trapezoidal rule

def integral_ext_trap(f,a,b,N):                     #funktioniert
    if b >= a:
        h = (b-a)/N
        x = a
        area = 0.5*(f(a)+f(b))
        i = 1
        while i < N:
            area += f(x+i*h)
            i += 1
        area = h*area
    else:
        h = (a-b)/N
        x = b
        area = 0.5*(f(a)+f(b))
        i = 1
        while i <= N:
            area += f(x+i*h)
            i += 1
        area = -h*area
    return area

In [14]:
order = 100000

I = s.integral_ext_trap(f,0,2,order)

exact = 3.5409610461819527733248015434794460

deviation = (exact-I)/exact

print("Thats the Integral: ", I)

print(deviation)

error = 1/(order**2)

print("This is the error: ", error)

Thats the Integral:  3.5409610469542954
-2.1811667064929294e-10
This is the error:  1e-10


In [15]:
#Extended Simpson-Rule

def integral_ext_simp(f,a,b,N):                         #funktioniert
    if b >= a:
        h = (b-a)/N
        x = a
        area = (1/3)*(f(a)+f(b))
        i = 1
        while i < N:
            if (i % 2 == 0):
                area += (2/3)*f(x+i*h)
            else:
                area += (4/3)*f(x+i*h)
            i += 1
        area = h*area
    else:
        h = (a-b)/N
        x = b
        area = (1/3)*(f(a)+f(b))
        i = 1
        while i < N:
            if (i % 2 == 0):
                area += (2/3)*f(x+i*h)
            else:
                area += (4/3)*f(x+i*h)
            i += 1
        area = h*area
    return area

In [16]:
order = 1000

I = s.integral_ext_simp(f,0,2,order)

exact = 3.5409610461819527733248015434794460

deviation = (exact-I)/exact

print("Thats the Integral: ", I)

print(deviation)

error = 1/(order**4)

print("This is the error: ", error)

Thats the Integral:  3.5409610461865273
-1.2918986938865157e-12
This is the error:  1e-12


In [17]:
#Romberg integration

def integral_romberg(f,a,b,eps):

    g = 0
    if b < a:
        z = a
        a = b
        b = z
        g = 1
    
    h = b - a 

    x = np.asfarray([h**2])
    y = np.asfarray([])

    Inew = 0
                                        
    I0 = (h/2)*(f(a)+f(b))
    y = np.append(y,I0)

    In = I0
    In1 = 0.5*(I0 + h*f(a+0.5*b))

    i = 2
    o = 0
    z = 0
    
    while o < 1:

        if np.abs((In1-In)) > np.abs(eps*In):
            k = 0
            sum = 0
            deltan = (h/(2**(i-1)))

            In = In1
            
            while k <= (2**(i-1)-1):
                sum += f(a+(k+0.5)*deltan)
                k += 1

            Inew = deltan*sum

            In1 = 0.5*(In+Inew)

            y = np.append(y,In1)
            i += 1
        else:
            o += 1
            
    for j in range(i-2):
        x = np.append(x,(h/(2*(j+1)))**2)

    I_final = y[np.size(y)-1]

    if g == 1:
        return -I_final,y

    return I_final,y
    

In [18]:
I,y = integral_romberg(f,0,2,1e-10)

exact = 3.5409610461819527733248015434794460

deviation = (exact-I)/exact

print("Thats the Integral: ", I)

print(deviation)

print("Number of Iterations: ", np.size(y))


Thats the Integral:  3.5409610462943344
-3.1737615316339727e-11
Number of Iterations:  18


In [19]:
#Romberg integration

def integral_romberg_extrapol(f,a,b,eps,order):

    g = 0
    if b < a:
        z = a
        a = b
        b = z
        g = 1

    h = b - a 

    x = np.asfarray([h**2])
    y = np.asfarray([])

    Inew = 0
                                        
    I0 = (h/2)*(f(a)+f(b))
    y = np.append(y,I0)

    In = I0
    In1 = 0.5*(I0 + h*f(a+0.5*b))

    i = 2
    

    while i < 3+order:
        k = 0
        sum = 0
        deltan = (h/(2**(i-1)))

        In = In1
        
        while k <= (2**(i-1)-1):
            sum += f(a+(k+0.5)*deltan)
            k += 1

        Inew = deltan*sum

        In1 = 0.5*(In+Inew)

        y = np.append(y,In1)
        x = np.append(x,(h/(2**i))**2)
        i += 1


    x1 = np.delete(x,0)
    y1 = np.delete(y,0)

    x = np.delete(x, np.size(x)-1)
    y = np.delete(y, np.size(y)-1)

    o = 0
    while o < 1:

        if np.abs(s.neville_algorithm(0,order,x1,y1)-s.neville_algorithm(0,order,x,y)) > np.abs(eps*s.neville_algorithm(0,order,x1,y1)):
            k = 0
            sum = 0
            deltan = (h/(2**(i-1)))

            In = In1
            
            while k <= (2**(i-1)-1):
                sum += f(a+(k+0.5)*deltan)
                k += 1

            Inew = deltan*sum

            In1 = 0.5*(In+Inew)

            x = x1
            y = y1
            y1 = np.append(y1,In1)
            x1 = np.append(x1,(h/(2**i))**2)
            x1 = np.delete(x1,0)
            y1 = np.delete(y1,0)

            i += 1
        else:
            if g == 1:
                return -y1[np.size(y1)-1],i
            else:
                return y1[np.size(y1)-1],i
    

In [20]:
I,i = integral_romberg_extrapol(f,0,2,1e-12,1)

exact = 3.5409610461819527733248015434794460

deviation = (exact-I)/exact

print("Thats the Integral: ", I)

print("Number of Iterations: ", i)

print(deviation)


Thats the Integral:  3.5409615065358757
Number of Iterations:  13
-1.3000818618954167e-07
